In [ ]:
library(ggplot2)
library(cowplot)
library(dplyr)
library(tibble)
library(AnnotationHub)
library(patchwork)
library(Seurat)
library(harmony)
library(magrittr)
library(SeuratWrappers)
library(gprofiler2)

Definitions of colors pallet

In [2]:
custom_colors <- list()

colors_dutch <- c(
  '#FFC312','#C4E538','#12CBC4','#FDA7DF','#ED4C67',
  '#F79F1F','#A3CB38','#1289A7','#D980FA','#B53471',
  '#EE5A24','#009432','#0652DD','#9980FA','#833471',
  '#EA2027','#006266','#1B1464','#5758BB','#6F1E51'
)

colors_spanish <- c(
  '#40407a','#706fd3','#f7f1e3','#34ace0','#33d9b2',
  '#2c2c54','#474787','#aaa69d','#227093','#218c74',
  '#ff5252','#ff793f','#d1ccc0','#ffb142','#ffda79',
  '#b33939','#cd6133','#84817a','#cc8e35','#ccae62',
  '#45aaf2', '#7f8c8d'
)

colors_turkish <- c(
  '#cd84f1','#ffcccc','#ff4d4d','#ffaf40','#fffa65',
  '#32ff7e','#7efff5','#18dcff','#7d5fff','#4b4b4b'
)

custom_colors$discrete <- c(colors_dutch, colors_spanish, colors_turkish)

custom_colors$cell_cycle <- setNames(
  c('#45aaf2', '#f1c40f', '#e74c3c', '#7f8c8d'),
  c('G1',      'S',       'G2M',     '-')
)

plot resolution

In [3]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [4]:
library(future)

In [5]:
plan("multicore", workers = 4)
plan()

multicore:
- args: function (..., workers = 4, envir = parent.frame())
- tweaked: TRUE
- call: plan("multicore", workers = 4)

Loading of the cell ranger matrix

In [58]:
SS_E11_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E11_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E11_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E11_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E12_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_A_arcref/outs/filtered_feature_bc_matrix")
SS_E12_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_B_arcref/outs/filtered_feature_bc_matrix")
SS_E12_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E12_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_A_arcref/outs/filtered_feature_bc_matrix")
SS_E13_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_C_arcref/outs/filtered_feature_bc_matrix")
SS_E13_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_D_arcref_2/outs/filtered_feature_bc_matrix")
SS_E13_E.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_E_arcref_2/outs/filtered_feature_bc_matrix")
SS_E13_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_C_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_D_arcref/outs/filtered_feature_bc_matrix")
SS_E14_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_A_arcref/outs/filtered_feature_bc_matrix")
SS_E14_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_B_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_C_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_D_arcref/outs/filtered_feature_bc_matrix")
SS_E15_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_A_arcref/outs/filtered_feature_bc_matrix")
SS_E15_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_B_arcref/outs/filtered_feature_bc_matrix")
SS_E15_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_C_arcref/outs/filtered_feature_bc_matrix")

In [ ]:
SS_E11_5_A <- CreateSeuratObject(counts = SS_E11_5_A.data)
SS_E11_5_B <- CreateSeuratObject(counts = SS_E11_5_B.data)
SS_E12_A <- CreateSeuratObject(counts = SS_E12_A.data)
SS_E13_5_B <- CreateSeuratObject(counts = SS_E13_5_B.data)
SS_E13_A <- CreateSeuratObject(counts = SS_E13_A.data)
SS_E13_D <- CreateSeuratObject(counts = SS_E13_D.data)
SS_E14_5_B <- CreateSeuratObject(counts = SS_E14_5_B.data)
SS_E14_A <- CreateSeuratObject(counts = SS_E14_A.data)
SS_E15_B <- CreateSeuratObject(counts = SS_E15_B.data)
SS_E12_5_A <- CreateSeuratObject(counts = SS_E12_5_A.data)
SS_E12_B <- CreateSeuratObject(counts = SS_E12_B.data)
SS_E13_5_C <- CreateSeuratObject(counts = SS_E13_5_C.data)
SS_E13_B <- CreateSeuratObject(counts = SS_E13_B.data)
SS_E13_E <- CreateSeuratObject(counts = SS_E13_E.data)
SS_E14_5_C <- CreateSeuratObject(counts = SS_E14_5_C.data)
SS_E14_B <- CreateSeuratObject(counts = SS_E14_B.data)
SS_E15_C <- CreateSeuratObject(counts = SS_E15_C.data)
SS_E12_5_B <- CreateSeuratObject(counts = SS_E12_5_B.data)
SS_E13_5_A <- CreateSeuratObject(counts = SS_E13_5_A.data)
SS_E13_5_D <- CreateSeuratObject(counts = SS_E13_5_D.data)
SS_E13_C <- CreateSeuratObject(counts = SS_E13_C.data)
SS_E14_5_D <- CreateSeuratObject(counts = SS_E14_5_D.data)
SS_E15_A <- CreateSeuratObject(counts = SS_E15_A.data)

Annotation of samples

In [60]:
SS_E11_5_A$SAMP <- "J11.5A"
SS_E11_5_B$SAMP <- "J11.5B"
SS_E12_A$SAMP <- "E12.A"
SS_E13_5_B$SAMP <- "E13.5B"
SS_E13_A$SAMP <- "E13.A"
SS_E13_D$SAMP <- "E13.D"
SS_E14_5_B$SAMP <- "E14.5B"
SS_E14_A$SAMP <- "E14.A"
SS_E15_B$SAMP <- "E15.B"
SS_E12_5_A$SAMP <- "E12.5.A"
SS_E12_B$SAMP <- "E12.B"
SS_E13_5_C$SAMP <- "E13.5.C"
SS_E13_B$SAMP <- "E13.B"
SS_E13_E$SAMP <- "E13.E"
SS_E14_5_C$SAMP <- "E14.5.C"
SS_E14_B$SAMP <- "E14.B"
SS_E15_C$SAMP <- "E14.C"
SS_E12_5_B$SAMP <- "E12.5.B"
SS_E13_5_A$SAMP <- "E13.5.A"
SS_E13_5_D$SAMP <- "E13.5.D"
SS_E13_C$SAMP <- "E13.C"
SS_E14_5_D$SAMP <- "E14.5.D"
SS_E15_A$SAMP <- "E15.A"

# QC : mitochondrial genes

In [62]:
# Connect to AnnotationHub
ah <- AnnotationHub()

# Access the Ensembl database for organism
ahDb <- query(ah, 
              pattern = c("Sus scrofa", "EnsDb"), 
              ignore.case = TRUE)

In [63]:
id <- ahDb %>%
  mcols() %>%
  rownames() %>%
  tail(n = 1)

In [64]:
edb <- ah[[id]]

loading from cache



Extract gene-level information from database

In [65]:
annotations <- genes(edb, 
                     return.type = "data.frame") 

Select annotations of interest

In [66]:
annotations <- annotations %>%
  dplyr::select(gene_id, gene_name, gene_biotype, seq_name, description, entrezid)

Extract IDs for mitochondrial genes

In [67]:
mt <- annotations %>%
        dplyr::filter(seq_name == "MT")
mt$gene_name <- ifelse(mt$gene_name == '', mt$gene_id, mt$gene_name)
mt <- mt$gene_name

Count Mt RNA percentage

In [68]:
SS_E11_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E11_5_B, slot = "counts")
SS_E11_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_A, slot = "counts")
SS_E12_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_B, slot = "counts")
SS_E13_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_A, slot = "counts")
SS_E13_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_D, slot = "counts")
SS_E13_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_5_B, slot = "counts")
SS_E14_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_A, slot = "counts")
SS_E14_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_B, slot = "counts")
SS_E15_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_5_A, slot = "counts")
SS_E12_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_B, slot = "counts")
SS_E12_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_C, slot = "counts")
SS_E13_5_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_B, slot = "counts")
SS_E13_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_E, slot = "counts")
SS_E13_E[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_5_C, slot = "counts")
SS_E14_5_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_B, slot = "counts")
SS_E14_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_C, slot = "counts")
SS_E15_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_5_B, slot = "counts")
SS_E12_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_A, slot = "counts")
SS_E13_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_D, slot = "counts")
SS_E13_5_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_C, slot = "counts")
SS_E13_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_5_D, slot = "counts")
SS_E14_5_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_A, slot = "counts")
SS_E15_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)

# merging of the different samples

In [70]:
embryo_gastrulation <- merge(x = SS_E11_5_A, y = c(SS_E11_5_B, SS_E12_A, SS_E12_B, SS_E12_5_A, SS_E12_5_B, SS_E13_A, SS_E13_B, SS_E13_C, SS_E13_D, SS_E13_E, SS_E13_5_A, SS_E13_5_B, SS_E13_5_C, SS_E13_5_D, SS_E14_A, SS_E14_B, SS_E14_5_B, SS_E14_5_C, SS_E14_5_D, SS_E15_A, SS_E15_B, SS_E15_C))

Warning message in CheckDuplicateCellNames(object.list = objects):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [71]:
gastrulation_metadata = read.csv('/home/adufour/work/gastrulation_data/pig_meta_data_new.csv',sep=',')

In [73]:
embryo_gastrulation <- subset(embryo_gastrulation, cells = gastrulation_metadata$cell_barcode)

In [76]:
rownames(gastrulation_metadata) <- gastrulation_metadata$cell_barcode

In [77]:
embryo_gastrulation@meta.data <- gastrulation_metadata[rownames(embryo_gastrulation@meta.data),]

In [81]:
saveRDS(embryo_gastrulation, file = "/home/adufour/work/rds_storage/gastrulation/embryo_E11_E15.rds")